In [3]:
from keras.models import Sequential
from keras.layers import Bidirectional, TimeDistributed, Dense, LSTM
import numpy as np
from keras.datasets import imdb
from keras.preprocessing import sequence


In [4]:
# Define functions to generate a sequence and multiple sequences
def get_sequence(n_timesteps):
    x = np.array([np.random.random() for _ in range(n_timesteps)])
    limit = n_timesteps / 4.0
    y = np.array([0 if val < limit else 1 for val in np.cumsum(x)])
    return x, y

def get_sequences(n_sequences, n_timesteps):
    seq_x, seq_y = list(), list()

    for _ in range(n_sequences):
        x, y = get_sequence(n_timesteps)
        seq_x.append(x)
        seq_y.append(y)

    seq_x = np.array(seq_x).reshape(n_sequences, n_timesteps, 1)
    seq_y = np.array(seq_y).reshape(n_sequences, n_timesteps, 1)
    
    return seq_x, seq_y


In [5]:
# Define the Bidirectional LSTM model
n_timesteps = 10
model = Sequential()
model.add(Bidirectional(LSTM(50, return_sequences=True), input_shape=(n_timesteps, 1)))
model.add(TimeDistributed(Dense(1, activation='sigmoid')))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_2 (Bidirecti  (None, 10, 100)           20800     
 onal)                                                           
                                                                 
 time_distributed_1 (TimeDi  (None, 10, 1)             101       
 stributed)                                                      
                                                                 
Total params: 20901 (81.64 KB)
Trainable params: 20901 (81.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [6]:
# Train the model
x_train, y_train = get_sequences(50000, n_timesteps)
model.fit(x_train, y_train, epochs=10, batch_size=10)



Epoch 1/10
5000/5000 [==============================] - 17s 3ms/step - loss: 0.0518 - accuracy: 0.9812
Epoch 2/10
5000/5000 [==============================] - 17s 3ms/step - loss: 0.0210 - accuracy: 0.9925
Epoch 3/10
5000/5000 [==============================] - 19s 4ms/step - loss: 0.0166 - accuracy: 0.9939
Epoch 4/10
5000/5000 [==============================] - 19s 4ms/step - loss: 0.0149 - accuracy: 0.9944
Epoch 5/10
5000/5000 [==============================] - 20s 4ms/step - loss: 0.0134 - accuracy: 0.9949
Epoch 6/10
5000/5000 [==============================] - 21s 4ms/step - loss: 0.0124 - accuracy: 0.9953
Epoch 7/10
5000/5000 [==============================] - 20s 4ms/step - loss: 0.0111 - accuracy: 0.9958
Epoch 8/10
5000/5000 [==============================] - 20s 4ms/step - loss: 0.0109 - accuracy: 0.9957
Epoch 9/10
5000/5000 [==============================] - 20s 4ms/step - loss: 0.0104 - accuracy: 0.9958
Epoch 10/10
5000/5000 [==============================] - 20s 4ms/step - l

In [7]:
# Evaluate the model
x_test, y_test = get_sequences(100, n_timesteps)
loss, accuracy = model.evaluate(x_test, y_test, verbose=1)
print(f'\nLoss: {loss}, Accuracy: {accuracy * 100}%')



4/4 [==============================] - 1s 3ms/step - loss: 0.0067 - accuracy: 0.9970

Loss: 0.0066572739742696285, Accuracy: 99.6999979019165%


In [8]:
# Make predictions
for _ in range(10):
    x_pred, y_actual = get_sequences(1, n_timesteps)
    y_pred = model.predict(x_pred, verbose=0)
    actual, pred = y_actual.reshape(n_timesteps), y_pred.reshape(n_timesteps)
    
    print(f'y={actual}, y_pred={pred}, correct={np.array_equal(actual, pred)}')


y=[0 0 0 0 1 1 1 1 1 1], y_pred=[1.3765856e-16 3.8709262e-13 3.9304462e-12 2.3285439e-04 9.9821472e-01
 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00], correct=False
y=[0 0 0 0 1 1 1 1 1 1], y_pred=[2.7037929e-15 1.0744926e-10 8.4953762e-16 1.1107689e-02 1.0000000e+00
 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00], correct=False
y=[0 0 0 0 0 0 1 1 1 1], y_pred=[7.1033493e-17 4.6542475e-16 9.1625936e-16 6.6485649e-16 1.1137375e-11
 1.6710980e-01 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00], correct=False
y=[0 0 0 0 0 0 1 1 1 1], y_pred=[3.3862719e-09 3.7834205e-13 6.2798741e-13 7.9580344e-12 3.1729383e-10
 1.0751907e-08 9.9990988e-01 1.0000000e+00 1.0000000e+00 1.0000000e+00], correct=False
y=[0 0 0 0 0 1 1 1 1 1], y_pred=[8.5687329e-17 1.8401999e-16 4.0032268e-14 3.5834714e-13 1.2174803e-01
 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00], correct=False
y=[0 0 0 0 0 0 1 1 1 1], y_pred=[1.2824681e-16 1.3

In [9]:
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense


vocab_size = 5000
max_sequence_length = 300

# Load the IMDB dataset
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)

# Pad sequences to a fixed length (necessary for input to LSTM)
x_train = sequence.pad_sequences(x_train, maxlen=max_sequence_length)
x_test = sequence.pad_sequences(x_test, maxlen=max_sequence_length)

# Create a Sequential model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_sequence_length))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

# Train the model
model.fit(x_train, y_train, epochs=3, batch_size=64, validation_split=0.2)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 300, 128)          640000    
                                                                 
 bidirectional_3 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 738945 (2.82 MB)
Trainable params: 738945 (2.82 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/3
313/313 [==============================] - 187s 592ms/step - loss: 0.4650 - accuracy: 0.7790 - val_loss: 0.3383 - val_accuracy: 0.8568
Epoch 2/3
313/313 [==============================] - 157s 502ms/step - 